In [36]:
import openai
import os

In [37]:
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())
openai.api_key  = os.getenv('OPENAI_API_KEY')
#MODEL="gpt-3.5-turbo"
MODEL="gpt-4"

In [38]:
def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

[](../images/03-Principle1.png)

## Prompting Principles
- **Principle 1: Write clear and specific instructions** (not short)
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [39]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

Provide clear, specific, and detailed instructions to guide a model towards the desired output, reducing the chances of irrelevant or incorrect responses, as longer prompts often offer more clarity and context.


![](../images/04-avoid-prompt-injection.png)

#### Tactic 2: Ask for a structured output
- JSON, HTML

In [40]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
  {
    "book_id": 1,
    "title": "Whispers of the Forgotten Realm",
    "author": "Aria Silverstone",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Quantum Paradox",
    "author": "Dr. Maxwell Straton",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "A Symphony of Shadows",
    "author": "Lilith Blackwood",
    "genre": "Mystery"
  }
]


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [42]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, you can add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [43]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [44]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The mighty oak stands tall amidst \ 
the fiercest storms, bending but never breaking; \ 
the delicate lotus blooms in murky waters, \ 
rising above adversity to share its beauty; \ 
the humble stone endures the test of time, \ 
weathering all elements with unwavering strength.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [45]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Jack and Jill, siblings in a charming village, embark on a quest to fetch water but both trip and fall, returning home with undimmed adventurous spirits.

Jack et Jill, frères et sœurs dans un charmant village, se lancent dans une quête pour chercher de l'eau mais trébuchent tous les deux, rentrant chez eux avec des esprits aventureux inaltérés.

Jack, Jill

{"french_summary": "Jack et Jill, frères et sœurs dans un charmant village, se lancent dans une quête pour chercher de l'eau mais trébuchent tous les deux, rentrant chez eux avec des esprits aventureux inaltérés.", "num_names": 2}
